In [0]:
# Firstly, we import the necessary libraries
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data
import torchvision
from torchvision import transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import cv2 
import pandas as pd
import os

In [0]:
# In order to acces google drive files and folders
from google.colab import drive
drive.mount('/content/gdrive');

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# The formatting of the csv file has already been done in the local pc
df = pd.read_csv('/content/gdrive/My Drive/driving_log.csv')

In [0]:
# Creation of a list consisting of the paths for the center, left and right camera images
center_path_list = df["center"].tolist()
left_path_list = df["left"].tolist()
right_path_list = df["right"].tolist()

In [0]:
# Creation of a list consisting of the steering angles for the center, left and right camera images
center_steering = df["steering_center"].tolist()
left_steering = df["steering_left"].tolist()
right_steering = df["steering_right"].tolist()

In [0]:
# Training set input label path will consist of 1000 images from each camera
training_set_paths = center_path_list[0:1000] + left_path_list[0:1000] + right_path_list[0:1000]
test_set_paths = center_path_list[1000:] + left_path_list[1000:] + right_path_list[1000:]

In [0]:
# Similarly we create lists for training and test set labels
training_set_labels = center_steering[0:1000] + left_steering[0:1000] + right_steering[0:1000]
for i in range(len(training_set_labels)):
  training_set_labels[i] = float(training_set_labels[i])
test_set_labels = center_steering[1000:] + left_steering[1000:] + right_steering[1000:]
for i in range(len(test_set_labels)):
  test_set_labels[i] = float(test_set_labels[i])

In [0]:
# Function to load the images stored in the lists (from the paths) 
def load_img(list_name):
  images = [];
  c = 0
  for path in list_name:
    c = c + 1
    print(c)
    img = cv2.imread(path)
    img = img[45:, 0:, 0:]
    img = cv2.resize(img, (200, 200), interpolation = cv2.INTER_AREA)
    images.append(img)
  return images

In [0]:
# Convert training set labels and test set labels into a numpy array
x = pd.Series(training_set_labels)
training_set_labels = x.to_numpy()
training_set_labels = np.float32(training_set_labels)
training_set_labels = torch.from_numpy(training_set_labels)

x = pd.Series(test_set_labels)
test_set_labels = x.to_numpy()
test_set_labels = np.float32(test_set_labels)
test_set_labels = torch.from_numpy(test_set_labels)

In [0]:
# Define the transformations that have to be applied on the input images and output labels
transform_img = transforms.Compose([
                                    transforms.ToTensor(), # input[channel] = (input[channel] - mean[channel]) / std[channel]
                                    transforms.Normalize(mean = [0.5, 0.5, 0.5], std = [0.5, 0.5, 0.5])
])

In [0]:
# We have to make custom classes in order to use the DataLoader. These classes inherit the Dataset class
from torch.utils.data import Dataset, DataLoader

class trainset(data.Dataset):
  def __init__(self, transform = None, train_path = None, train_label_path = None):
    self.train_img = load_img(train_path)
    self.transform = transform
    self.label = train_label_path

  def __len__(self):
    return len(self.train_img)
  
  def __getitem__(self, index):
    img = self.transform(self.train_img[index])
    label = self.label[index]
    return img, label

class testset(data.Dataset):
  def __init__(self, transform = None, test_path = None, test_label_path = None):
    self.test_img = load_img(test_path)
    self.transform = transform
    self.label = test_label_path

  def __len__(self):
    return len(self.test_img)
  
  def __getitem__(self, index):
    img = self.transform(self.test_img[index])
    label = self.label[index]
    return img, label

test_dataset = testset(transform_img, test_set_paths, test_set_labels)
train_dataset = trainset(transform_img, training_set_paths, training_set_labels)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [0]:
# Load the train and test data
trainloader = data.DataLoader(train_dataset, batch_size = 2, shuffle = True, num_workers = 4)
testloader = data.DataLoader(test_dataset, batch_size = 2, shuffle = True, num_workers = 4)

In [0]:
# Designing the network
class Network(nn.Module):
  # class Network will inherit nn.Module and its methods
  def __init__(self):
    super().__init__()
    # super().__init__() will call the constructor of nn.Module. The parameters of the methods
    # that we will use later (of nn class) will then get initialized
    self.conv1 = nn.Conv2d(3, 16, 3)
    self.conv2 = nn.Conv2d(16, 32, 3)
    self.conv3 = nn.Conv2d(32, 64, 3)
    self.conv4 = nn.Conv2d(64, 128, 3)
    self.pool = nn.MaxPool2d(2, 2)
    self.b1 = nn.BatchNorm2d(16)
    self.b2 = nn.BatchNorm2d(32)
    self.b3 = nn.BatchNorm2d(64)
    self.b4 = nn.BatchNorm2d(128)
    self.fc1 = nn.Linear(10 * 10 * 128, 100)
    self.fc2 = nn.Linear(100, 50)
    self.fc3 = nn.Linear(50, 10)
    self.fc4 = nn.Linear(10, 1)
    self.dr = nn.Dropout2d(0.5)

  def forward(self, x):
    x = self.pool(self.b1(F.relu(self.conv1(x))))
    x = self.pool(self.b2(F.relu(self.conv2(x))))
    x = self.pool(self.b3(F.relu(self.conv3(x))))
    x = self.pool(self.b4(F.relu(self.conv4(x))))
    x = x.view(-1, 10 * 10 * 128)
    x = F.relu(self.dr(self.fc1(x)))
    x = F.relu(self.dr(self.fc2(x)))
    x = F.relu(self.dr(self.fc3(x)))
    x = F.relu(self.dr(self.fc4(x)))
    return x

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"); # use gpu if gpu is available else use cpu
print(device)
network = Network()
network.to(device)

cuda:0


Network(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (conv4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (b1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (b4): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fc1): Linear(in_features=12800, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=50, bias=True)
  (fc3): Linear(in_features=50, out_features=10, bias=True)
  (fc4): Linear(in_features=10, out_features=1, bias=True)
  (dr): Dropout2d(p=0.5, inplace=False)
)

In [0]:
# Defining the loss function criterion and optimizer
criterion = nn.MSELoss() # Used in NVIDIA paper
optimizer = optim.Adam(network.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False) # Momentum is generally taken as 0.9
# When net.parameters() is called, automatically requires_grad is set True for the parameters in order to calculate 
# their gradients

In [0]:
# Training the data
for epoch in range(200):
  for i, data in enumerate(trainloader):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad(); # Zero the parameter gradeints so that in the next cycle, those values don't create
    # trouble
    outputs = network.forward(inputs)
    if (outputs.size() == torch.Size([2, 1])):
      outputs = outputs.reshape(2)
    else:
      outputs = outputs.reshape(1)
    loss = criterion(outputs, labels)
    optimizer.step(); # This leads to parameter update
    if i % 1000 == 0:
      print(loss.item()); # Prints the scalar value of loss

0.10026299953460693
0.08000201731920242
0.1600000113248825
0.0012500000884756446
0.2943991720676422
0.08115975558757782
0.14124999940395355
0.02399780973792076
0.21124997735023499
0.2565021514892578
0.08000000566244125
0.682457447052002
0.06985588371753693
0.7979675531387329
0.12500005960464478
0.002141976961866021
0.08000000566244125
0.27193859219551086
0.36299118399620056
0.1056966483592987
0.030619779601693153
0.08000000566244125
0.08040426671504974
0.1360328495502472
0.07950437813997269
0.4816662669181824
0.32391971349716187
0.08353042602539062
0.796942949295044
0.08500000834465027
0.10624999552965164
0.1600000113248825
0.14798526465892792
0.096525639295578
0.08000000566244125
0.20214195549488068
0.11975331604480743
0.44635894894599915
0.00564905721694231
0.06217164173722267
0.5112808346748352
0.020079556852579117
0.1258746236562729
0.09651819616556168
0.1600000113248825
0.698204517364502
0.04625000059604645
0.12054474651813507
0.28625351190567017
0.119681216776371
0.02697454020380

KeyboardInterrupt: ignored

In [0]:
PATH = '/content/gdrive/My Drive/model.h5';
torch.save(network.state_dict(),PATH); # A state_dict is simply a Python dictionary 
# object that maps each layer to its parameter tensor. 

In [2]:
import torch 
print(torch.__version__)


1.3.1
